In [6]:
from google.colab import files
import shutil
import os

# Upload model folder zip first or upload files individually
uploaded = files.upload()  # Upload all files from emotion_model folder and the Excel file

# If you uploaded a zip of the model folder, unzip it here
# !unzip emotion_model.zip -d emotion_model

# Or if you uploaded files individually, move them into a folder
model_folder = "emotion_model"
os.makedirs(model_folder, exist_ok=True)
for filename in uploaded.keys():
    if filename != "Headlines 22 Apr to 16 May.xlsx":  # skip Excel file here
        shutil.move(filename, os.path.join(model_folder, filename))


Saving Peacetime news.xlsx to Peacetime news.xlsx
Saving config.json to config.json
Saving model.safetensors to model.safetensors
Saving special_tokens_map.json to special_tokens_map.json
Saving tokenizer_config.json to tokenizer_config.json
Saving vocab.txt to vocab.txt


In [7]:
!pip install safetensors

from transformers import BertTokenizer, BertForSequenceClassification
import torch, numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = "./emotion_model"

tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()

emotion_cols = ["Anger","Fear","Joy","Sadness","Trust","Surprise",
                "Disgust","Anticipation","Nostalgia","Pride","Shame","Vindication"]

best_thresholds = {'Anger':0.50,'Fear':0.45,'Joy':0.25,'Sadness':0.50,'Trust':0.50,
                   'Surprise':0.15,'Disgust':0.25,'Anticipation':0.10,'Nostalgia':0.10,
                   'Pride':0.15,'Shame':0.40,'Vindication':0.40}

def predict_emotions(headline):
    inp = tokenizer(headline, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        logits = model(**inp).logits.cpu().numpy()[0]
    probs = 1/(1+np.exp(-logits))
    return {emo: round(p,3) for emo,p in zip(emotion_cols, probs)
            if p >= best_thresholds[emo]}


In [ ]:
import pandas as pd

df = pd.read_excel("Headlines 22 Apr to 16 May.xlsx")
headlines = df["Headline"].tolist()

records = [predict_emotions(h) for h in headlines]
emotion_df = pd.DataFrame(records).fillna(0)
emotion_df.insert(0, "Headline", headlines)

out_name = "emotion_predictions.csv"
emotion_df.to_csv(out_name, index=False)

print(f"Processed {len(headlines)} headlines.")


In [ ]:
from google.colab import files
files.download(out_name)


In [11]:
import pandas as pd

# Load Excel file
df = pd.read_excel("emotion_model/Peacetime news.xlsx", header=None)
df.columns = ["Headline"]  # rename the single column

# Predict emotions for each headline
emotion_preds = df["Headline"].astype(str).apply(predict_emotions)

# Convert prediction dicts to separate columns
emotion_df = emotion_preds.apply(pd.Series).fillna(0)

# Combine with original headlines
output_df = pd.concat([df, emotion_df], axis=1)

# Save to new Excel file
output_df.to_excel("Peacetime_with_emotions.xlsx", index=False)


KeyboardInterrupt: 

In [12]:
from google.colab import files
files.download("Peacetime_with_emotions.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>